<a href="https://colab.research.google.com/github/chohyungrae/-AI-Peer-school-/blob/master/seq2seq%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참조사이트; https://wikidocs.net/24996

**글자 레벨 기계 번역기(Character-Level Neural Machine Translation) 구현하기**

> (참조; Keras의 시퀀스-투-시퀀스 학습)

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

이중 언어 문장 쌍 download;http://www.manythings.org/anki/
에서 fra-eng.zip 파일 다운로드

In [ ]:
'''
해당 데이터를 읽고 전처리를 진행해보겠습니다. 
앞으로의 코드에서 src는 source의 줄임말로 입력 문장을 나타내며, 
tar는 target의 줄임말로 번역하고자 하는 문장을 나타냅니다.
'''

In [ ]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
#전체 샘플의 개수는 총 약 17만 7천개입니다.
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
len(lines)

In [ ]:
'''
해당 데이터는 약 17만 7천개의 병렬 문장 샘플로 구성되어있지만 
여기서는 간단히 60,000개의 샘플만 가지고 기계 번역기를 구축하려고함.
우선 전체 데이터 중 60,000개의 샘플만 저장하고 
현재 데이터가 어떤 구성이 되었는지 확인해보겠습니다.
'''

lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

In [ ]:
'''
위의 테이블은 랜덤으로 선택된 10개의 샘플을 보여줍니다. 
번역 문장에 해당되는 프랑스어 데이터는 시작을 의미하는 심볼 <sos>과 
종료를 의미하는 심볼 <eos>을 넣어주어야 합니다. 

여기서는 <sos>와 <eos> 대신 '\t'를 시작 심볼, '\n'을 종료 심볼로 간주하여 추가하고 
다시 데이터를 출력해보겠습니다.
'''

lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)



In [ ]:
'''
위에서 랜덤으로 10개의 샘플을 선택하여 출력하였습니다. 
프랑스어 데이터에서 시작 심볼과 종료 심볼이 추가된 것을 볼 수 있습니다. 

이제 글자 집합을 생성해보겠습니다. 
단어 집합이 아니라 글자 집합이라고 하는 이유는 
토큰 단위가 단어가 아니라 글자이기 때문입니다.
'''

# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [ ]:
#글자 집합의 크기를 보겠습니다.
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

In [ ]:
'''
위와 같이 영어와 프랑스어는 각각 약 80개와 100개의 글자가 존재합니다. 
이 중에서 인덱스를 임의로 부여하여 일부만 출력해봅시다. 
정렬하여 순서를 정해준 뒤에 인덱스를 사용하여 출력
'''

src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

In [ ]:
'''
위와 같이 글자 집합에 글자 단위로 저장된 것을 확인할 수 있습니다. 
이제 각 글자에 인덱스를 부여하겠습니다.
'''

src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

In [ ]:
'''
인덱스가 부여된 글자 집합으로부터 갖고있는 훈련 데이터에 정수 인코딩을 수행하겠습니다. 
우선 인코더의 입력이 될 영어 문장 샘플에 대해서 정수 인코딩을 수행해보고, 5개의 샘플을 출력해봅시다.
'''

encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

In [ ]:
'''
위와 같이 정수 인코딩이 수행된 것을 볼 수 있습니다. 

이제 디코더의 입력이 될 프랑스어 데이터에 대해서 정수 인코딩을 수행해보겠습니다.
'''

decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

In [ ]:
'''
위와같이 정상적으로 정수 인코딩이 수행된 것을 볼 수 있습니다. 

아직 정수 인코딩을 수행해야 할 데이터가 하나 더 남았습니다. 
디코더의 예측값과 비교하기 위한 실제값이 필요합니다. 
그런데 이 실제값에는 시작 심볼에 해당되는 <sos>가 있을 필요가 없습니다. 
그래서 정수 인코딩 과정에서 <sos>를 제거합니다. 
즉, 모든 프랑스어 문장의 맨 앞에 붙어있는 '\t'를 제거하도록 합니다.
'''

decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

In [ ]:
'''
위의 결과값은 앞서 먼저 만들었던 디코더의 입력값에 해당되는 decoder_input 데이터와 
비교하면 decoder_input에서는 모든 문장의 앞에 붙어있던 
숫자 1이 decoder_target에서는 제거된 것을 볼 수 있습니다. 
'\t'가 인덱스가 1이므로 정상적으로 제거된 것입니다. 

이제 모든 데이터에 대해서 정수 인덱스로 변경하였으니 패딩 작업을 수행하겠습니다. 
패딩을 위해서 영어 문장과 프랑스어 문장 각각에 대해서 가장 길이가 긴 샘플의 길이를 알아보겠습니다.
'''

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

In [ ]:
'''
각각 25와 76의 길이를 가집니다. 
병렬 데이터는 영어와 프랑스어의 길이는 하나의 쌍이라고 하더라도 전부 다르므로 
패딩을 할 때도 이 두 개의 데이터의 길이를 전부 동일하게 맞춰줄 필요는 없습니다. 
영어 데이터는 영어 샘플들끼리, 프랑스어는 프랑스어 샘플들끼리 길이를 맞추어서 
패딩하면 됩니다. 
여기서는 가장 긴 샘플의 길이에 맞춰서 영어 데이터의 샘플은 전부 길이가
 25가 되도록 패딩하고, 프랑스어 데이터의 샘플은 전부 길이가 76이 되도록 패딩합니다.
'''

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [ ]:
'''
이제 모든 값에 대해서 원-핫 인코딩을 수행합니다. 
글자 단위 번역기므로 워드 임베딩은 별도로 사용되지 않으며, 
예측값과의 오차 측정에 사용되는 실제값뿐만 아니라 입력값도 원-핫 벡터를 사용하겠습니다.
'''

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

여기까지
데이터에 대한 전처리가 모두 끝났습니다. 본격적으로 seq2seq 모델을 설계해보겠습니다.

# **교사 강요(Teacher forcing)**

현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다고 설명하였는데 decoder_input이 왜 필요할까요?

훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용할 겁니다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 합니다. 이런 상황이 반복되면 훈련 시간이 느려집니다. 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있습니다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 합니다.

In [ ]:
# seq2seq 기계 번역기 훈련시키기
# seq2seq 모델을 설계하고 교사 강요를 사용하여 훈련시켜보도록 하겠습니다.

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [ ]:
'''
위를 보면
LSTM의 은닉 상태 크기는 256으로 선택하였습니다. 
인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 
return_state=True로 설정합니다. 이제 인코더에 입력을 넣으면 내부 상태를 리턴합니다.

LSTM에서 state_h, state_c를 리턴받는데, 이는 각각 LSTM 챕터에서 배운 
은닉 상태와 셀 상태에 해당됩니다.  LSTM은 은닉 상태와 셀 상태라는 
두 가지 상태를 가진다.
즉, 단지 은닉 상태만 전달하는 게 아니라 은닉 상태와 셀 상태 두 가지를 전달한다고 
생각하면 됩니다. 이 두 가지 상태를 encoder_states에 저장합니다. 
encoder_states를 디코더에 전달하므로서 이 두 가지 상태 모두를 
디코더로 전달합니다. 이것을 컨텍스트 벡터라고 함
'''

In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
'''
디코더는 인코더의 마지막 은닉 상태를 초기 은닉 상태로 사용합니다. 
위에서 initial_state의 인자값으로 encoder_states를 주는 코드가 이에 해당됩니다. 
또한 동일하게 디코더의 은닉 상태 크기도 256으로 주었습니다. 
디코더도 은닉 상태, 셀 상태를 리턴하기는 하지만 훈련 과정에서는 사용하지 않습니다. 
그 후 출력층에 프랑스어의 단어 집합의 크기만큼 뉴런을 배치한 후 
소프트맥스 함수를 사용하여 실제값과의 오차를 구합니다.
'''

In [ ]:
'''
입력으로는 인코더 입력과 디코더 입력이 들어가고, 
디코더의 실제값인 decoder_target도 필요합니다. 
배치 크기는 64로 하였으며 총 50 에포크를 학습합니다. 
여기서는 seq2seq의 메커니즘과 짧은 문장과 긴 문장에 대한 성능 차이에 
대한 확인을 중점으로 두고 훈련 데이터에 과적합 된 상태로 동작 단계로 넘어갑니다.
'''

model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
750/750 [==============================] - 418s 558ms/step - loss: 0.4062 - val_loss: 0.4766
Epoch 2/50
750/750 [==============================] - 419s 558ms/step - loss: 0.3489 - val_loss: 0.4301
Epoch 3/50
750/750 [==============================] - 417s 555ms/step - loss: 0.3133 - val_loss: 0.4018
Epoch 4/50
750/750 [==============================] - 416s 554ms/step - loss: 0.2885 - val_loss: 0.3765
Epoch 5/50
750/750 [==============================] - 418s 557ms/step - loss: 0.2705 - val_loss: 0.3620
Epoch 6/50
750/750 [==============================] - 420s 561ms/step - loss: 0.2565 - val_loss: 0.3511
Epoch 7/50
750/750 [==============================] - 417s 556ms/step - loss: 0.2448 - val_loss: 0.3428
Epoch 8/50
750/750 [==============================] - 417s 556ms/step - loss: 0.2352 - val_loss: 0.3379
Epoch 9/50
750/750 [==============================] - 416s 554ms/step - loss: 0.2268 - val_loss: 0.3323
Epoch 10/50
750/750 [==============================] - 418s 557m

**seq2seq 기계 번역기 동작시키기**
seq2seq는 훈련할 때와 동작할 때의 방식이 다르다.
따라서 입력한 문장에 대해서 기계 번역을 하도록 모델을 조정하고 동작시켜보도록 하겠습니다.

전체적인 번역 동작 단계를 정리하면 
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
2. 상태와 <SOS>에 해당하는 '\t'를 디코더로 보냅니다.
3. 디코더가 <EOS>에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.

In [ ]:
'''
우선 인코더를 정의합니다. 
encoder_inputs와 encoder_states는 훈련 과정에서 이미 정의한 것들을 재사용
'''

encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
# 이제 디코더를 설계해보겠습니다.

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
'''
단어로부터 인덱스를 얻는 것이 아니라 인덱스로부터 단어를 얻을 수 있는 
index_to_src와 index_to_tar를 만들었습니다.
'''

index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

지금까지 글자 단위의 seq2seq를 구현하였습니다.